# 相関係数を求めるデモ
サンプルデータを用意し、PostgreSQLとMADlibそれぞれを使って相関係数を求める
## 前準備

In [321]:
# ipython-sqlをロード
%reload_ext sql
# 文字コードをUTF-8にセット
# (データベースの文字コードもUTF-8にセットしておくこと)
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

## PostgreSQLに接続＆接続確認

In [322]:
%sql postgresql://postgres@centos72/postgres
%sql SELECT version();

version
"PostgreSQL 9.6.1 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-4), 64-bit"


## MADlibが正常にインストールされているか確認

In [304]:
%sql SELECT madlib.version();

version
"MADlib version: 1.10.0-dev, git revision: rel/v1.9.1-8-g82e56a4, cmake configuration time: 2016年 11月 22日 火曜日 00:25:23 UTC, build type: RelWithDebInfo, build system: Linux-3.10.0-327.el7.x86_64, C compiler: gcc 4.8.5, C++ compiler: g++ 4.8.5"


## サンプルデータ準備

In [325]:
%%sql

DROP TABLE IF EXISTS
    pearson_test
;

CREATE TABLE
    pearson_test(
         "ItemID" INT
        ,"User1" NUMERIC
        ,"User2" NUMERIC
        ,"User3" NUMERIC
        ,"User4" NUMERIC
        ,"User5" NUMERIC
    )
;

INSERT INTO
    pearson_test
VALUES
     (101, 5.0, 2.0, 2.5, 5.0, 4.0)
    ,(102, 3.0, 4.4, 2.0, 4.0, 3.0)
    ,(103, 2.5, 5.0, 2.0, 3.0, 2.0)
    ,(104, 4.0, 1.0, 4.0, 4.5, 4.0)
    ,(105, 2.5, 5.0, 2.5, 2.0, 3.5)
    ,(106, 2.5, 4.5, 3.0, 4.0, 5.0)
    ,(107, 2.5, 1.0, 5.0, 3.5, 2.0)
;

SELECT * FROM pearson_test;

ItemID,User1,User2,User3,User4,User5
101,5.0,2.0,2.5,5.0,4.0
102,3.0,4.4,2.0,4.0,3.0
103,2.5,5.0,2.0,3.0,2.0
104,4.0,1.0,4.0,4.5,4.0
105,2.5,5.0,2.5,2.0,3.5
106,2.5,4.5,3.0,4.0,5.0
107,2.5,1.0,5.0,3.5,2.0


## PostgreSQL本体の相関係数関数を使って特定のユーザ同士の相関係数を求める

In [318]:
%sql SELECT corr("User1","User4") FROM pearson_test;

corr
0.769711105935


In [319]:
%sql SELECT corr("User2","User3") FROM pearson_test;

corr
-0.794656321349


## MADlibの相関係数関数を使ってUser1から5を指定して相関係数を求める

In [327]:
%%sql

DROP TABLE IF EXISTS
     pearson_test_output
    ,pearson_test_output_summary
;

SELECT 
    madlib.correlation(
         'pearson_test'
        ,'pearson_test_output'
        ,'
             "User1"
            ,"User2"
            ,"User3"
            ,"User4"
            ,"User5"
         '
    )
;

SELECT * FROM pearson_test_output;

column_position,variable,User1,User2,User3,User4,User5
1,User1,1.0,None,None,None,None
2,User2,-0.546929577339,1.0,None,None,None
3,User3,-0.0377195470233,-0.794656321349,1.0,None,None
4,User4,0.769711105935,-0.582961865158,0.149966529064,1.0,None
5,User5,0.364518243178,0.0260498161861,-0.100966206016,0.410984968443,1.0
